적대적 탐색 알고리즘의 동작 방식을 이해하고 이를 이용하여 게임 프로그램을 만드는 예를 살펴 봅시다. 여기 제공하는 코드는 GitHub aima-python의 코드를 기반으로 일부 수정한 것임.

In [1]:
import random
import math
infinity = math.inf

## 게임 문제 정의
자신만의 게임 문제를 정의하려면 Game 클래스의 서브 클래스를 정의하면 됨.

In [2]:
class Game:
    """문제를 정의하기 위한 Problem 클래스에 대응되는 게임 정의를 위한 클래스.
    경로 비용과 목표 검사 대신 각 상태에 대한 효용 함수와 종료 검사로 구성됨.
    게임을 정의하려면 이 클래스의 서브 클래스를 만들어서
    actions, result, is_terminal, utility를 구현하면 됨.
    필요에 따라 게임의 초기 상태를 지정하려면,
    클래스 생성자에서 초기 상태를 initial 에 세팅하면 됨."""

    def actions(self):
        """허용 가능한 수(move) 리스트"""
        raise NotImplementedError

    def result(self, move):
        """수(move)를 두었을 때의 결과 상태 리턴"""
        raise NotImplementedError

    def is_terminal(self):
        """종료 상태이면 True 리턴"""
        return not self.actions()
    
    def utility(self):
        """게임이 종료됐을 때 효용 함수 값"""
        raise NotImplementedError

In [3]:
def play_game(game, strategies: dict, verbose=False):
    """번갈아 가면서 두는 게임 진행 함수.
    strategies: {참가자 이름: 함수} 형태의 딕셔너리. 
    여기서 함수(game)는 참가자의 수(move)를 찾는 함수"""
    while not game.is_terminal():
        player = game.to_move
        move = strategies[game.to_move](game)
        if move:
            game = game.result(move)
        else:
            game.turn_change()
        if verbose: 
            print('Player', player, 'move:', move)
            game.display()
    return game

### 알파-베타 탐색

In [4]:
def alphabeta_search(game, null):
    """알파-베타 가지치기를 사용하여 최고의 수를 결정하기 위한 게임 트리 탐색."""



    def max_value(game, alpha, beta):
        if game.is_terminal():
            return game.utility(), None
        v, move = -infinity, None
        for a in game.actions():
            v2, _ = min_value(game.result(a), alpha, beta)
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        if not move:
            game.to_move = '○'
            return min_value(game, alpha, beta)
        return v, move

    def min_value(game, alpha, beta):
        if game.is_terminal():
            return game.utility(), None
        v, move = +infinity, None
        for a in game.actions():
            v2, _ = max_value(game.result(a), alpha, beta)
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            if v <= alpha:
                return v, move
        if not move:
            game.to_move = '●'
            return max_value(game, alpha, beta)   
        return v, move
    
    if(game.to_move=='○'):
        return min_value(game, -infinity, +infinity)
    else :
        return max_value(game, -infinity, +infinity)

### 휴리스틱 알파-베타 탐색에서 필요한 함수

In [5]:
def cutoff_depth(d):
    """깊이 d까지만 탐색하도록 하는 중단 함수: depth > d이면 True 리턴."""
    return lambda game, depth: depth > d

def h(game):
    """터미널 노드까지 도달하지 않고 평가함수를 반환하기 위해, 오셀로 판의 각 모서리에 가중치를 부여
    판의 각 꼭짓점 => +
    판의 모든 모서리 => +
    판의 각 꼭짓점을 둘러싼 세 점 => +
    판의 각 꼭짓점이 채워지지 않은 상태에서 각 꼭짓점을 둘러싼 세 점 => - """
    
    cnt = 0
    for i in range(game.height):
        for j in range(game.width):
            if game.board[i][j] == '●':
                cnt += 1
            elif game.board[i][j]=='○':
                cnt -= 1

            if i==game.height-1 or i==0 or j==game.width-1 or j==0 :
                if game.board[i][j]=='●':
                    cnt += 1
                elif game.board[i][j]=='○':
                    cnt -= 1
            
    corners = [[[0,0],[[1,0],[1,1],[0,1]]],
              [[0,game.width-1],[[1,game.width-1],[1,game.width-2],[0,game.width-2]]],
              [[game.height-1,0],[[game.height-2,0],[game.height-2,1],[game.height-1,0]]],
              [[game.height-1,game.width-1],[[game.height-2,game.width-1],[game.height-2,game.width-2],[game.height-1,game.width-2]]]]
    
    for corner in corners:
        if(game.board[corner[0][0]][corner[0][1]]=='●'): cnt+=3
        elif(game.board[corner[0][0]][corner[0][1]]=='○'):cnt-=3
        else: 
            for xy in corner[1]:
                if(game.board[xy[0]][xy[1]]=='●'): cnt-=2
                elif(game.board[xy[0]][xy[1]]=='○'):cnt+=2
    return cnt

### 휴리스틱 알파-베타 탐색

In [6]:
def h_alphabeta_search(game, c_depth):
    """휴리스틱 알파-베타 탐색"""
    cutoff=cutoff_depth(c_depth)
    player = game.to_move

    def max_value(game, alpha, beta, depth):
        if game.is_terminal():
            return game.utility(), None
        if cutoff(game, depth):
            return h(game), None
        v, move = -infinity, None
        for a in game.actions():
            v2, _ = min_value(game.result(a), alpha, beta, depth+1)
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        if not move:
            game.to_move = '○'
            return min_value(game, alpha, beta, depth)
        return v, move

    def min_value(game, alpha, beta, depth):
        if game.is_terminal():
            return game.utility(), None
        if cutoff(game, depth):
            return h(game), None
        v, move = +infinity, None
        for a in game.actions():
            v2, _ = max_value(game.result(a), alpha, beta, depth+1)
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            if v <= alpha:
                return v, move
        if not move:
            game.to_move =  '●'
            return max_value(game, alpha, beta, depth)   
        return v, move

    if(game.to_move=='○'):
        return min_value(game, -infinity, +infinity, 0)
    else :
        return max_value(game, -infinity, +infinity, 0)


## Othello(리버시) 게임

### 게임 정의

In [7]:
class Othello(Game):
    """Othello 게임. 보드 크기: height * width. 
    게임 종료시 더 많은 색을 가진 사람이 승리
    '●'와 '○'가 게임 플레이. '●'가 먼저 플레이.
    (0, 0) 위치는 보드의 좌상단 끝 위치."""

    def __init__(self, height, width, to_move):
        self.height=height
        self.width=width
        self.board = [[ '.' for x in range(width)] for y in range(height)]
        self.board[width//2][height//2]='●'
        self.board[width//2-1][height//2-1]='●'
        self.board[width//2-1][height//2]='○'
        self.board[width//2][height//2-1]='○'
        self.to_move = to_move

    def actions(self):
        move = ((0,1),(1,0),(0,-1),(-1,0),(1,1),(-1,1),(1,-1),(-1,-1))
        """내 돌을 놓을 수 있는 좌표를 담은 배열 반환"""
        action = []
        for i in range(self.height):
            for j in range(self.width):
                for d in move:
                    if(self.board[i][j]=='.' and self.feasible(i+d[0], j+d[1], d) > 0):
                        action.append([i,j])
                        break
        return action

    def feasible(self, x, y, dxy): 
        """해당 위치에 돌을 놓을 수 있는지 확인하는 함수"""
        if(x < 0 or y < 0 or x >= self.height or y >= self.width or self.board[x][y] == '.'):
            return -1
        elif(self.board[x][y] == self.to_move):
            return 0
        else:
            tmp = self.feasible(x + dxy[0], y + dxy[1], dxy)
            if(tmp != -1):
                return tmp + 1
            return -1
    
    def turn(self, x, y, dxy):
        """상대 돌을 뒤집는 함수"""
        if(x < 0 or y < 0 or x>=self.height or y>=self.width or self.board[x][y]=='.'):
            return -1
        elif(self.board[x][y]==self.to_move):
            return 0
        else:
            tmp = self.turn(x+dxy[0], y+dxy[1], dxy)
            if(tmp != -1):
                self.board[x][y] = self.to_move
                return 0
            return -1
    
    def check_all_dirt(self,action):
        """8방향에 대해 돌을 놓을 수 있는지 확인하는 함수"""
        move = ((0,1),(1,0),(0,-1),(-1,0),(1,1),(-1,1),(1,-1),(-1,-1))
        
        for d in move:
            self.turn(action[0]+d[0], action[1]+d[1], d)

    def result(self, action):
        """반환할 새로운 게임(보드)를 위해 새로운 객체 board2생성
        깊은복사를 위해 새로운 보드에 현재 보드(2차원 배열) 복사
        돌을 놓고 바뀐 결과를 담은 객체 board2반환"""
        board2 = Othello(self.height, self.width, self.to_move)
        board2.board = [self.board[x][:] for x in range(self.height)]
        board2.check_all_dirt(action)
        board2.board[action[0]][action[1]] = self.to_move
        board2.turn_change()
        
        return board2

    def turn_change(self):
        """수를 놓는 차례 변경"""
        if self.to_move=='○': 
            self.to_move = '●'
        else: 
            self.to_move = '○'

    def utility(self):
        """●가 먼저 시작
        반환되는 값이 양수면 ●승리
        반환되는 값이 음수면 ○승리"""
        cnt = 0
        for i in range(self.height):
            for j in range(self.width):
                if self.board[i][j] == '●':
                    cnt += 1
                elif self.board[i][j] =='○':
                    cnt -= 1
        return cnt

    def is_terminal(self):
        """두 플레이어 모두 수를 둘 수 없을 때, 터미널 상태로 인지하고 중단"""
        a = self.actions()
        self.turn_change()
        b = self.actions()
        self.turn_change()
        if(not a and not b):
            return 1
        return 0

    def display(self, util = False): 
        """현재 배열 출력 함수"""
        for i in range(self.height):
            for j in range(self.width):
                if(self.board[i][j]=='.'): print('',end=" ")
                print(self.board[i][j], end=" ")
            print()
        if util:
            return self.utility()

### 게임 참가자(player) 정의

In [8]:
def random_player(game):
    """허용되는 수(move) 중에서 무작위로 하나를 선택하는 플레이어"""
    act = game.actions()
    if not act:
        return None
    return random.choice(game.actions())

def player(search_algorithm, c_depth=0):
    """지정된 탐색 알고리즘을 사용하는 플레이어: (game)을 입력 받아 move를 리턴하는 함수."""
    return lambda game : search_algorithm(game, c_depth)[1]

In [9]:
def query_player(game):
    """다음 수(move)를 직접 입력하는 형태의 플레이어"""
    cheerup = ["좋습니다.", "허를 찌르는 수입니다.", "훌륭하군요!", "한 수 배워갑니다.", "예상치 못한 수...(사실 그런거 없음)", "대단합니다!"]
    print("현재 상태:")
    game.display()
    print("")
    move = None
    actions = game.actions()
    if actions:
        while True:
            print(f"가능한 수: {actions}")
            move_string = input('당신의 수는? (돌을 둘 위치 입력; 예: (1,1)(세로, 가로)): ')
            col, row = int(move_string.split(',')[0]), int(move_string.split(',')[1])
            if [col, row] in actions:
                print()
                print(random.choice(cheerup))
                print()
                break
                
            print()
            print("===============================")
            print("가능한 수만 둘 수 있습니다! 다시 두세요")
            print("===============================")
            print()
            print("현재 상태:")
            game.display()
            continue
        try:
            move = eval(move_string)
            
        except NameError:
            move = move_string
    else:
        print("=================================")
        print('가능한 수가 없음: 상대방에게 순서가 넘어감.')
        print("=================================")
    return move

# 게임하기

#### 가능한 탐색 : 

##### 1. player(  alphabeta_search  )  => 알파-베타 가지치기 탐색

##### 2. player(  h_alphabeta_search ,   cutoff_depth  )  => 휴리스틱 알파-베타 탐색 ( 매개변수로 원하는 cutoff 깊이 입력 )

##### 3. random_player  => 랜덤 탐색(무작위 보행)

##### 4. query_player  => 사용자 플레이
<hr>

# 플레이 방법: 


```python
양식: play_game(Othello(보드판의 높이, 보드판의 너비, '●'), {'●':선수자 탐색, '○':후수자 탐색}, verbose=참 or 거짓) 
```

        위 양식에 변수를 입력

        보드판의 높이: 게임을 할 보드판의 높이
        보드판의 너비: 게임을 할 보드판의 너비
        선수자 탐색 : 가능한 탐색 4가지중 한가지 선택
        후수자 탐색 : 가능한 탐색 4가지중 한가지 선택
        참 or 거짓 : True입력시 진행 상황마다 보드판 출력, False입력시 출력x

#### ※주의사항※

        보드의 판을 5x5이상으로 설정 할 경우, alphabeta_search가 매우 느려져서 결과 확인에 큰 무리가 있음
        보드의 판을 4x4로 설정 할 경우, 후수자가 모서리를 확보하는 경우의 수가 많아지기 때문에 후수자의 승률이 매우 올라감
        보드의 판을 5x5로 설정 할 경우, h_alphabeta_search의 cutoff 깊이는 6~8이 적당함 ( 그 이상은 실행 속도가 매우 느려짐 )
        보드의 판을 6x6으로 설정 할 경우, h_alphabeta_search의 cutoff 깊이는 4~6이 적당함 ( 그 이상은 실행 속도가 매우 느려짐 )
        
<hr>

### 1. 게임 결과만 확인하기

###### 기본 세팅 :  양식에 원하는 변수 입력
    
    방법1: 양식에 .utility()를 붙여서 실행 => 게임 결과 값만 출력
          출력된 값 => 양수이면 선수자 승리, 음수이면 후수자 승리, 0이면 무승부
          (결과의 절댓값이 클수록 월등히 이긴것임)
          
    방법2: 양식에 .display()를 붙여서 실행 => 게임 결과 보드판만 출력
    
    방법3: 양식에 .display(True)를 붙여서 실행 => 게임 결과 보드판 출력 및 값 반환
<hr>

##### 게임 결과만 확인하기; 방법1

In [10]:
play_game(Othello(4, 4, '●'), {'●':player(h_alphabeta_search, 6), '○':player(alphabeta_search)}, verbose=False).utility()

-9

##### 게임 결과만 확인하기; 방법2

In [11]:
play_game(Othello(4, 4, '●'), {'●':player(alphabeta_search), '○':random_player}, verbose=False).display()

○ ○ ● ● 
 . ● ● ● 
● ● ● ● 
 . ●  . ● 


##### 게임 결과만 확인하기; 방법3

In [12]:
print(play_game(Othello(4, 4, '●'), {'●':player(h_alphabeta_search, 4), '○':player(h_alphabeta_search, 6)}, verbose=False).display(True))

● ○ ○ ○ 
 . ○ ○ ○ 
 . ○ ○ ○ 
● ○ ○ ● 
-8


<hr>

### 2. n번 실행 후 통계 확인하기

###### 기본세팅 :  반복문의 첫 문장( 10행 ) 에 위치한 양식에 원하는 변수 입력  ( 양식 끝에 있는 utility()함수는 지우지 말것! )
    
    원하는 반복 횟수를 입력하면 사용자가 실행시킨 탐색 방법으로 진행한 게임의 통계 확인 가능

In [13]:
draw = 0
win = 0
lose = 0
n = int(input("몇 회 실행하시겠습니까?"))
print("시작", end = '')
print(" "*(n-4), end = '')
print("종료")

for i in range(n):
  ex = play_game(Othello(10, 10, '●'), {'●':random_player, '○':player(h_alphabeta_search, 1)}, verbose=False).utility()
  if ex > 0:
    win += 1
  elif ex < 0:
    lose += 1
  else:
    draw += 1
  
  if i == n-1:
    print("진행 완료!")
    print()
  else:
    print("#", end = '')

print(f"{n}회 실행시 통계")
print("======================")
print(f"선수자 승률: {round(win/n*100, 2)}%")
print(f"후수자 승률: {round(lose/n*100, 2)}%")
print(f"무승부 횟수: {draw}회")

몇 회 실행하시겠습니까?5
시작 종료
####진행 완료!

5회 실행시 통계
선수자 승률: 0.0%
후수자 승률: 100.0%
무승부 횟수: 0회


<hr>

### 3. 사용자 플레이하기

###### 기본세팅 :  첫 행에 위치한 양식에 원하는 변수 입력  ( 양식 끝에 있는 utility()함수는 지우지 말것! )

        탐색 방법을 넣는 변수중 한개에 query_player를 입력
        다른 한개에는 대결하기 원하는 탐색 방법 입력 후 실행
        
        게임 종료시 어떤 플레이어가 이겼는지 반환

In [14]:
score = play_game(Othello(4, 4, '●'), {'●':query_player, '○':player(alphabeta_search)}, verbose = False).display(True)

if score > 0:
    print("선수자의 승리입니다!")
elif score < 0:
    print("후수자의 승리입니다!")
else:
    print("무승부 입니다!")
print("재시작을 원한다면 Shift + Enter를 눌러주세요")

현재 상태:
 .  .  .  . 
 . ● ○  . 
 . ○ ●  . 
 .  .  .  . 

가능한 수: [[0, 2], [1, 3], [2, 0], [3, 1]]
당신의 수는? (돌을 둘 위치 입력; 예: (1,1)(세로, 가로)): 0,2

허를 찌르는 수입니다.

현재 상태:
 .  . ● ○ 
 . ● ○  . 
 . ○ ●  . 
 .  .  .  . 

가능한 수: [[1, 3], [2, 0], [3, 1]]
당신의 수는? (돌을 둘 위치 입력; 예: (1,1)(세로, 가로)): 1,3

예상치 못한 수...(사실 그런거 없음)

현재 상태:
 . ○ ○ ○ 
 . ○ ● ● 
 . ○ ●  . 
 .  .  .  . 

가능한 수: [[0, 0], [1, 0], [2, 0], [3, 0]]
당신의 수는? (돌을 둘 위치 입력; 예: (1,1)(세로, 가로)): 0,0

대단합니다!

현재 상태:
● ○ ○ ○ 
 . ● ○ ○ 
 . ○ ○ ○ 
 .  .  .  . 

가능한 수: [[3, 1], [3, 3]]
당신의 수는? (돌을 둘 위치 입력; 예: (1,1)(세로, 가로)): 3,1

한 수 배워갑니다.

현재 상태:
● ○ ○ ○ 
 . ○ ○ ○ 
○ ○ ○ ○ 
 . ●  .  . 

가능한 수: [[3, 3]]
당신의 수는? (돌을 둘 위치 입력; 예: (1,1)(세로, 가로)): 3,3

훌륭하군요!

현재 상태:
● ○ ○ ○ 
 . ● ○ ○ 
○ ○ ○ ○ 
 . ● ○ ● 

가능한 수가 없음: 상대방에게 순서가 넘어감.
현재 상태:
● ○ ○ ○ 
 . ● ○ ○ 
○ ○ ○ ○ 
○ ○ ○ ● 

가능한 수가 없음: 상대방에게 순서가 넘어감.
● ○ ○ ○ 
○ ○ ○ ○ 
○ ○ ○ ○ 
○ ○ ○ ● 
후수자의 승리입니다!
재시작을 원한다면 Shift + Enter를 눌러주세요
